# Painel do lojista 
## Dashboard e acompanhamento de suas vendas, com seção de recomendações de melhorias para suas publicações.

### Breve descrição

Este notebook cria uma base de dados para um marketplace. Aqui encontrará o modelo lógico da base de dados, representando a parte transacional 
do projeto.

Vamos trabalhar com a premissa de usar 2 replicas deste banco. Uma de escrita (master) e uma slave (de leitura).

A replica slave será usada para realização de consultas mais pesadas para extração dos dados e inseridos num banco de dados de documento. Assim sendo possível 
    ter uma melhor performance para dados mais consolidados;

### Criar banco marketplace_db e inserir dados

In [ ]:
-- resolver problemas em tempo de desenvolvimento
DROP DATABASE IF EXISTS `marketplace_db`;

Query OK

In [5]:
-- Criação do Banco de Dados
CREATE DATABASE IF NOT EXISTS `marketplace_db`;
USE `marketplace_db`;

-- -----------------------------------------------------
-- Tabela `lojistas`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `lojistas` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `email` VARCHAR(255) NOT NULL UNIQUE,
  `dados_cadastrais` VARCHAR(255) NULL,
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `clientes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `clientes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `email` VARCHAR(255) NOT NULL UNIQUE,
  `perfil` VARCHAR(45) NOT NULL DEFAULT 'padrão',
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `categorias`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `categorias` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `produtos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `produtos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `preco` DECIMAL(10,2) NOT NULL,
  `estoque` INT NOT NULL DEFAULT 0,
  `lojista_id` INT NOT NULL,
  `descricao` TEXT(4000) NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_produtos_lojistas`
    FOREIGN KEY (`lojista_id`)
    REFERENCES `lojistas` (`id`)
);

-- -----------------------------------------------------
-- Tabela associativa `produtos_categorias`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `produtos_categorias` (
  `produto_id` INT NOT NULL,
  `categoria_id` INT NOT NULL,
  PRIMARY KEY (`produto_id`, `categoria_id`),
  CONSTRAINT `fk_prodcat_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`),
  CONSTRAINT `fk_prodcat_categorias`
    FOREIGN KEY (`categoria_id`)
    REFERENCES `categorias` (`id`)
);

-- -----------------------------------------------------
-- Tabela `pedidos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `pedidos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `cliente_id` INT NOT NULL,
  `data` DATETIME NOT NULL,
  `status` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_pedidos_clientes`
    FOREIGN KEY (`cliente_id`)
    REFERENCES `clientes` (`id`)
);

-- -----------------------------------------------------
-- Tabela `itens_pedidos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `itens_pedidos` (
  `pedido_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `quantidade` INT NOT NULL,
  `preco_unitario` DECIMAL(10,2) NOT NULL,
  PRIMARY KEY (`pedido_id`, `produto_id`),
  CONSTRAINT `fk_itens_pedidos`
    FOREIGN KEY (`pedido_id`)
    REFERENCES `pedidos` (`id`),
  CONSTRAINT `fk_itens_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `Avaliacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `avaliacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `cliente_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `nota` INT NOT NULL,
  `comentario` TEXT NULL,
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_avaliacoes_clientes`
    FOREIGN KEY (`cliente_id`)
    REFERENCES `clientes` (`id`),
  CONSTRAINT `fk_avaliacoes_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `precos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `precos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `produto_id` INT NOT NULL,
  `valor` DECIMAL(10,2) NOT NULL,
  `tipo_preco` VARCHAR(45) NOT NULL,
  `data_inicio_vigencia` DATE NOT NULL,
  `data_fim_vigencia` DATE NULL,
  `ativo` BOOLEAN NOT NULL,
  `data_criacao` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_precos_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `reclamacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `reclamacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `pedido_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `titulo` VARCHAR(255) NOT NULL,
  `texto` TEXT NULL,
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_reclamacoes_pedidos`
    FOREIGN KEY (`pedido_id`)
    REFERENCES `pedidos` (`id`),
  CONSTRAINT `fk_reclamacoes_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `transacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `transacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `lojista_id` INT NOT NULL,
  `valor` DECIMAL(10,2) NOT NULL,
  `tipo` VARCHAR(45) NOT NULL,
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_transacoes_lojistas`
    FOREIGN KEY (`lojista_id`)
    REFERENCES `lojistas` (`id`)
);


Query OK

In [ ]:
-- Criar inserts aqui

## Adicionando registros

In [ ]:
-- Lojistas (2 registros)
INSERT INTO `Lojistas` (`nome`, `email`, `dados_cadastrais`) VALUES
('TecnoLoja Ltda', 'contato@tecnoloja.com', 'CNPJ: 11.222.333/0001-44'),
('Casa & Cia', 'vendas@casaecia.com', 'CNPJ: 55.666.777/0001-88');

-- Clientes (20 registros)
INSERT INTO `Clientes` (`nome`, `email`, `perfil`) VALUES
('João Silva', 'joao.silva@email.com', 'padrão'), ('Maria Oliveira', 'maria.oliveira@email.com', 'vip'),
('Carlos Pereira', 'carlos.p@email.com', 'padrão'), ('Ana Costa', 'ana.costa@email.com', 'padrão'),
('Pedro Martins', 'pedro.m@email.com', 'vip'), ('Sandra Ferreira', 'sandra.f@email.com', 'padrão'),
('Lucas Gomes', 'lucas.g@email.com', 'padrão'), ('Juliana Rodrigues', 'juliana.r@email.com', 'padrão'),
('Gabriel Almeida', 'gabriel.a@email.com', 'vip'), ('Beatriz Barbosa', 'beatriz.b@email.com', 'padrão'),
('Marcos Souza', 'marcos.s@email.com', 'padrão'), ('Fernanda Lima', 'fernanda.l@email.com', 'padrão'),
('Rafael Nunes', 'rafael.n@email.com', 'padrão'), ('Camila Santos', 'camila.s@email.com', 'vip'),
('Tiago Mendes', 'tiago.m@email.com', 'padrão'), ('Larissa Castro', 'larissa.c@email.com', 'padrão'),
('Bruno Ribeiro', 'bruno.r@email.com', 'padrão'), ('Patrícia Rocha', 'patricia.r@email.com', 'padrão'),
('Felipe Jesus', 'felipe.j@email.com', 'padrão'), ('Amanda Dias', 'amanda.d@email.com', 'vip');

-- Categorias (5 registros)
INSERT INTO `Categorias` (`nome`) VALUES
('Eletrônicos'), ('Móveis'), ('Eletrodomésticos'), ('Informática'), ('Decoração');

-- Produtos (20 registros)
INSERT INTO `Produtos` (`nome`, `preco`, `estoque`, `lojista_id`) VALUES
('Smartphone X', 2999.90, 50, 1), ('Notebook Pro', 7499.50, 30, 1), ('Smart TV 50"', 3200.00, 40, 1),
('Fone Bluetooth', 499.99, 150, 1), ('Cadeira Gamer', 1200.00, 25, 1), ('Teclado Mecânico', 350.00, 80, 1),
('Mouse sem Fio', 150.00, 200, 1), ('Monitor 27" 4K', 2800.00, 35, 1), ('Webcam Full HD', 450.00, 60, 1),
('Impressora Multifuncional', 899.00, 45, 1), ('Sofá 3 Lugares', 2500.00, 20, 2), ('Mesa de Jantar', 1800.00, 15, 2),
('Guarda-Roupa Casal', 2200.00, 18, 2), ('Geladeira Frost Free', 3800.00, 22, 2), ('Fogão 5 Bocas', 1500.00, 30, 2),
('Micro-ondas', 750.00, 50, 2), ('Lava e Seca 11kg', 4200.00, 12, 2), ('Luminária de Chão', 380.50, 40, 2),
('Conjunto de Panelas', 550.00, 60, 2), ('Tapete Sala 2x3m', 480.00, 25, 2);

-- ProdutosCategorias (25 registros)
INSERT INTO `ProdutosCategorias` (`produto_id`, `categoria_id`) VALUES
(1, 1), (2, 1), (2, 4), (3, 1), (3, 3), (4, 1), (5, 2), (5, 4), (6, 4),
(7, 4), (8, 4), (9, 4), (10, 4), (11, 2), (12, 2), (13, 2), (14, 3),
(15, 3), (16, 3), (17, 3), (18, 5), (19, 3), (20, 5), (1, 4), (18, 2);

-- Pedidos (20 registros)
INSERT INTO `Pedidos` (`cliente_id`, `data`, `status`) VALUES
(1, '2025-06-01 10:00:00', 'Entregue'), (2, '2025-06-02 11:30:00', 'Enviado'),
(3, '2025-06-03 14:00:00', 'Processando'), (4, '2025-06-04 09:00:00', 'Entregue'),
(5, '2025-06-05 18:00:00', 'Enviado'), (6, '2025-06-06 20:00:00', 'Entregue'),
(7, '2025-06-07 22:15:00', 'Cancelado'), (8, '2025-06-08 12:00:00', 'Entregue'),
(9, '2025-06-09 13:45:00', 'Processando'), (10, '2025-06-10 16:00:00', 'Enviado'),
(11, '2025-06-11 17:20:00', 'Entregue'), (12, '2025-06-12 10:10:00', 'Entregue'),
(13, '2025-06-13 08:00:00', 'Enviado'), (14, '2025-06-14 11:00:00', 'Processando'),
(15, '2025-06-15 15:30:00', 'Entregue'), (16, '2025-06-16 19:00:00', 'Enviado'),
(17, '2025-06-17 21:00:00', 'Entregue'), (18, '2025-06-18 23:00:00', 'Entregue'),
(19, '2025-06-19 09:30:00', 'Processando'), (20, '2025-06-20 14:00:00', 'Enviado');

-- ItensPedidos (25 registros)
INSERT INTO `ItensPedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(1, 1, 1, 2999.90), (1, 4, 2, 499.99), (2, 11, 1, 2500.00), (3, 2, 1, 7499.50),
(4, 15, 1, 1500.00), (4, 16, 1, 750.00), (5, 5, 1, 1200.00), (6, 20, 1, 480.00),
(8, 14, 1, 3800.00), (9, 3, 1, 3200.00), (10, 18, 2, 380.50), (11, 7, 3, 150.00),
(12, 12, 1, 1800.00), (13, 8, 1, 2800.00), (14, 17, 1, 4200.00), (15, 6, 1, 350.00),
(16, 19, 1, 550.00), (17, 9, 1, 450.00), (18, 10, 1, 899.00), (19, 13, 1, 2200.00),
(20, 1, 1, 2999.90), (20, 6, 1, 350.00), (2, 19, 2, 550.00), (5, 7, 2, 150.00),
(11, 4, 1, 499.99);

-- Transacoes (20 registros)
INSERT INTO `Transacoes` (`lojista_id`, `valor`, `tipo`, `data`) VALUES
(1, 3999.89, 'Venda', '2025-06-01 10:05:00'), (2, 2500.00, 'Venda', '2025-06-02 11:35:00'),
(1, 7499.50, 'Venda', '2025-06-03 14:05:00'), (2, 2250.00, 'Venda', '2025-06-04 09:05:00'),
(1, 1200.00, 'Venda', '2025-06-05 18:05:00'), (2, 480.00, 'Venda', '2025-06-06 20:05:00'),
(2, 3800.00, 'Venda', '2025-06-08 12:05:00'), (1, 3200.00, 'Venda', '2025-06-09 13:50:00'),
(2, 761.00, 'Venda', '2025-06-10 16:05:00'), (1, 450.00, 'Venda', '2025-06-11 17:25:00'),
(2, 1800.00, 'Venda', '2025-06-12 10:15:00'), (1, 2800.00, 'Venda', '2025-06-13 08:05:00'),
(2, 4200.00, 'Venda', '2025-06-14 11:05:00'), (1, 350.00, 'Venda', '2025-06-15 15:35:00'),
(2, 550.00, 'Venda', '2025-06-16 19:05:00'), (1, 450.00, 'Venda', '2025-06-17 21:05:00'),
(1, 899.00, 'Venda', '2025-06-18 23:05:00'), (2, 2200.00, 'Venda', '2025-06-19 09:35:00'),
(1, 3349.90, 'Venda', '2025-06-20 14:05:00'), (2, -500.00, 'Taxa Mensal', '2025-06-30 23:59:59');

-- Avaliacoes (20 registros)
INSERT INTO `Avaliacoes` (`cliente_id`, `produto_id`, `nota`, `comentario`, `data`) VALUES
(1, 1, 5, 'Excelente produto!', '2025-06-05 10:00:00'),
(1, 4, 4, 'Bom, mas a bateria poderia durar mais.', '2025-06-05 10:01:00'),
(4, 15, 2, 'Fogão veio com um batedor quebrado.', '2025-06-08 14:00:00'),
(4, 16, 3, 'Apenas ok.', '2025-06-08 14:01:00'),
(6, 20, 1, 'Qualidade muito ruim, desbotou na primeira semana.', '2025-06-10 09:00:00'),
(8, 14, 5, 'Geladeira espaçosa e silenciosa. Adorei!', '2025-06-12 18:00:00'),
(11, 7, 5, 'Mouse muito preciso.', '2025-06-15 11:00:00'),
(12, 12, 4, 'Bonita, mas difícil de montar.', '2025-06-16 12:00:00'),
(15, 6, 2, 'As teclas são muito barulhentas.', '2025-06-19 13:00:00'),
(17, 9, 4, 'Boa imagem, mas o software é um pouco lento.', '2025-06-21 15:00:00'),
(18, 10, 5, 'Funciona perfeitamente.', '2025-06-22 16:00:00'),
(2, 11, 4, 'Confortável, mas o tecido esquenta.', '2025-06-07 10:00:00'),
(3, 2, 5, 'Máquina incrível para trabalho.', '2025-06-08 11:00:00'),
(5, 5, 2, 'Não é tão confortável quanto parece.', '2025-06-10 12:00:00'),
(9, 3, 5, 'Imagem fantástica!', '2025-06-14 14:00:00'),
(10, 18, 3, 'A luz é mais fraca do que o esperado.', '2025-06-15 15:00:00'),
(13, 8, 4, 'Ótimo monitor, cores vibrantes.', '2025-06-18 18:00:00'),
(14, 17, 5, 'Superou minhas expectativas.', '2025-06-19 19:00:00'),
(16, 19, 1, 'O teflon descascou muito rápido.', '2025-06-21 20:00:00'),
(20, 1, 5, 'Melhor celular que já tive!', '2025-06-25 21:00:00');

-- Reclamacoes (5 registros de exemplo)
INSERT INTO `Reclamacoes` (`cliente_id`, `produto_id`, `titulo`, `texto`, `data`) VALUES
(4, 15, 'Produto com defeito', 'O fogão chegou com um dos queimadores amassado e não funciona.', '2025-06-09 09:30:00'),
(6, 20, 'Produto de baixa qualidade', 'O tapete começou a soltar fios e desbotar com menos de um mês de uso.', '2025-06-11 10:00:00'),
(15, 6, 'Produto barulhento', 'O teclado é vendido como silencioso, mas faz muito barulho ao digitar.', '2025-06-20 11:00:00'),
(16, 19, 'Teflon de péssima qualidade', 'As panelas já estão descascando com pouquíssimo uso. Impossível cozinhar.', '2025-06-22 14:00:00'),
(5, 5, 'Cadeira desconfortável', 'A descrição prometia conforto, mas a cadeira é dura e causa dor nas costas.', '2025-06-11 15:00:00');

-- Precos (20 registros)
INSERT INTO `Precos` (`ProdutoID`, `Valor`, `TipoPreco`, `DataInicioVigencia`, `DataFimVigencia`, `Ativo`) VALUES
(1, 3299.90, 'Base', '2025-01-01', NULL, 0), (1, 2999.90, 'Promocional', '2025-05-20', '2025-07-20', 1),
(2, 7999.00, 'Base', '2025-01-01', NULL, 0), (2, 7499.50, 'Promocional', '2025-06-01', '2025-06-30', 1),
(3, 3200.00, 'Base', '2025-03-01', NULL, 1),
(4, 499.99, 'Base', '2025-02-01', NULL, 1),
(5, 1200.00, 'Base', '2025-04-01', NULL, 1),
(6, 399.00, 'Base', '2025-01-01', NULL, 0), (6, 350.00, 'Queima de Estoque', '2025-05-01', NULL, 1),
(11, 2800.00, 'Base', '2025-01-01', NULL, 0), (11, 2500.00, 'Promocional', '2025-06-01', '2025-07-01', 1),
(14, 4100.00, 'Base', '2025-02-01', NULL, 0), (14, 3800.00, 'Oferta', '2025-05-15', '2025-06-15', 1),
(15, 1650.00, 'Base', '2025-02-01', NULL, 0), (15, 1500.00, 'Oferta', '2025-05-15', '2025-06-15', 1),
(20, 520.00, 'Base', '2025-03-01', NULL, 0), (20, 480.00, 'Promocional', '2025-06-01', '2025-06-10', 1),
(19, 600.00, 'Base', '2025-01-01', NULL, 0), (19, 550.00, 'Oferta', '2025-06-15', '2025-06-25', 1),
(17, 4200.00, 'Base', '2025-04-10', NULL, 1);

### 1. Total de Vendas por Lojista

In [ ]:
SELECT
    l.lojista_id ID,
    l.nome AS Lojista,
    SUM(ip.quantidade * ip.preco_unitario) AS TotalVendido
FROM
    Lojistas l
JOIN
    Produtos p ON l.lojista_id = p.lojista_id
JOIN
    ItensPedidos ip ON p.produto_id = ip.produto_id
GROUP BY
    l.nome
ORDER BY
    TotalVendido DESC;

### 2. Top 10 Produtos Mais Vendidos

In [ ]:
SELECT
    p.nome AS Produto,
    SUM(ip.quantidade) AS QuantidadeVendida
FROM
    ItensPedidos ip
JOIN
    Produtos p ON ip.produto_id = p.produto_id
GROUP BY
    p.produto_id, p.nome
ORDER BY
    QuantidadeVendida DESC
LIMIT 10;

### 3. Alerta de Estoque Baixo

In [ ]:
SELECT
    p.nome AS Produto,
    p.estoque,
    l.nome AS Lojista
FROM
    Produtos p
JOIN
    Lojistas l ON p.lojista_id = l.lojista_id
WHERE
    p.estoque < 30
ORDER BY
    p.estoque ASC;

### 4. Avaliação Média dos Produtos com Baixo Desempenho

In [ ]:
SELECT
    l.lojista_id,
    p.nome AS Produto,
    l.nome AS Lojista,
    AVG(a.nota) AS MediaDeAvaliacoes,
    COUNT(a.avaliacao_id) AS QuantidadeDeAvaliacoes
FROM
    Produtos p
JOIN
    Avaliacoes a ON p.produto_id = a.produto_id
JOIN
    Lojistas l ON p.lojista_id = l.lojista_id
GROUP BY
    p.produto_id, p.nome, l.nome
HAVING
    MediaDeAvaliacoes < 3
ORDER BY
    MediaDeAvaliacoes ASC;

### 5. Clientes Mais Valiosos (Top 5)

In [ ]:
SELECT
    c.nome AS Cliente,
    c.email,
    SUM(ip.quantidade * ip.preco_unitario) AS TotalGasto
FROM
    Clientes c
JOIN
    Pedidos pd ON c.cliente_id = pd.cliente_id
JOIN
    ItensPedidos ip ON pd.pedido_id = ip.pedido_id
GROUP BY
    c.cliente_id, c.nome, c.email
ORDER BY
    TotalGasto DESC
LIMIT 5;